In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv', encoding='latin-1')

In [ ]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [ ]:
# Selecting   the 'tweet_text' and 'is_there_an_emotion_directed_at_a_brand_or_product' columns    for sentiment analysis

data=data[['tweet_text','is_there_an_emotion_directed_at_a_brand_or_product']]

In [ ]:
data.columns = ['tweet', 'sentiment']

In [ ]:
data.shape

(9093, 2)

In [ ]:
pd.set_option('display.max_colwidth',None)


In [ ]:
data.head()

,tweet,sentiment
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [ ]:
data['sentiment'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: sentiment, dtype: int64

In [ ]:
# preprocessing

In [ ]:
data['tweet'].isnull().sum()

1

In [ ]:
data['tweet'].fillna('', inplace=True)

In [ ]:
# feature extraction

In [ ]:
x = data['tweet']
y = data['sentiment']

In [ ]:
y

0                         Negative emotion
1                         Positive emotion
2                         Positive emotion
3                         Negative emotion
4                         Positive emotion
                       ...                
9088                      Positive emotion
9089    No emotion toward brand or product
9090    No emotion toward brand or product
9091    No emotion toward brand or product
9092    No emotion toward brand or product
Name: sentiment, Length: 9093, dtype: object

In [ ]:
# encoding

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [ ]:
y

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [ ]:
# Tokenizing  using Keras

In [ ]:
from keras.preprocessing import text
tokenizer=text.Tokenizer()
tokenizer.fit_on_texts(list(data['tweet']))
tokenized_text=tokenizer.texts_to_sequences(data['tweet'])

In [ ]:
len(tokenized_text[0])

24

In [ ]:
len(tokenized_text[1])

22

In [ ]:
from keras.utils import pad_sequences
x=pad_sequences(tokenized_text,maxlen=100)

In [ ]:
# train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# SimpleRNN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SimpleRNN,Dropout

In [ ]:
len(tokenizer.word_index)

10147

In [ ]:
# Create a Sequential model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))

# Add a SimpleRNN layer with 32 units

model.add(SimpleRNN(32))
model.add(Dropout(0.5))

# Add a Dense layer with 50 units
#model.add(Dense(50,activation = 'relu'))
#model.add(Dropout(0.5))

# Add the final Dense layer with 4 units (for 4 classes)
model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1298944   
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                5152      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 4)                 132       
                                                                 
Total params: 1304228 (4.98 MB)
Trainable params: 1304228 (4.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 23s 79ms/step - loss: 0.9720 - accuracy: 0.5545 - val_loss: 0.8513 - val_accuracy: 0.6223
Epoch 2/10
205/205 [==============================] - 12s 56ms/step - loss: 0.7556 - accuracy: 0.7062 - val_loss: 0.8735 - val_accuracy: 0.6126
Epoch 3/10
205/205 [==============================] - 11s 52ms/step - loss: 0.5106 - accuracy: 0.8266 - val_loss: 0.9502 - val_accuracy: 0.6236
Epoch 4/10
205/205 [==============================] - 19s 94ms/step - loss: 0.3359 - accuracy: 0.8909 - val_loss: 1.0459 - val_accuracy: 0.6264
Epoch 5/10
205/205 [==============================] - 11s 52ms/step - loss: 0.2694 - accuracy: 0.9143 - val_loss: 1.1542 - val_accuracy: 0.5838
Epoch 6/10
205/205 [==============================] - 10s 48ms/step - loss: 0.2319 - accuracy: 0.9221 - val_loss: 1.2384 - val_accuracy: 0.5989
Epoch 7/10
205/205 [==============================] - 11s 55ms/step - loss: 0.2033 - accuracy: 0.9294 - val_loss: 1.2592 - val_accuracy:

In [ ]:
y_pred=model.predict(x_test)

57/57 [==============================] - 1s 9ms/step


In [ ]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 1s 9ms/step - loss: 1.3677 - accuracy: 0.6053
Test Accuracy: 60.53%


In [ ]:
#LSTM

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))
model.add(LSTM(32))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 22s 95ms/step - loss: 0.9204 - accuracy: 0.6075 - val_loss: 0.8359 - val_accuracy: 0.6113
Epoch 2/10
205/205 [==============================] - 18s 88ms/step - loss: 0.6818 - accuracy: 0.7240 - val_loss: 0.8154 - val_accuracy: 0.6538
Epoch 3/10
205/205 [==============================] - 18s 86ms/step - loss: 0.4816 - accuracy: 0.8203 - val_loss: 0.8521 - val_accuracy: 0.6470
Epoch 4/10
205/205 [==============================] - 17s 83ms/step - loss: 0.3621 - accuracy: 0.8660 - val_loss: 0.9355 - val_accuracy: 0.6772
Epoch 5/10
205/205 [==============================] - 18s 86ms/step - loss: 0.2828 - accuracy: 0.8912 - val_loss: 1.0071 - val_accuracy: 0.6525
Epoch 6/10
205/205 [==============================] - 18s 88ms/step - loss: 0.2296 - accuracy: 0.9131 - val_loss: 1.0275 - val_accuracy: 0.6497
Epoch 7/10
205/205 [==============================] - 19s 91ms/step - loss: 0.1896 - accuracy: 0.9203 - val_loss: 1.2435 - val_accuracy:

In [ ]:
y_pred=model.predict(x_test)

57/57 [==============================] - 1s 16ms/step


In [ ]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 1s 20ms/step - loss: 1.5808 - accuracy: 0.6592
Test Accuracy: 65.92%
